In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [ ]:
import warnings
import math
import os
import sys
import logging
from pathlib import Path
import panel as pn
import numpy as np
import pandas as pd
import param
import bokeh
from copy import deepcopy
import datetime
from pytz import timezone
import lzma
import pickle
import yaml
import json
import socket
import time
from contextlib import redirect_stdout
from pathlib import Path
from collections import OrderedDict
from tempfile import TemporaryDirectory, NamedTemporaryFile
import hashlib
import shutil

import sys
from conda.exceptions import EnvironmentLocationNotFound
from conda.gateways.disk.test import is_conda_environment
from conda.cli.main_list import print_packages

In [ ]:
from astropy.time import Time, TimeDelta
from zoneinfo import ZoneInfo
import matplotlib as mpl
import matplotlib.pyplot as plt
import hvplot.pandas

In [ ]:
import lsst.resources

In [ ]:
import rubin_scheduler
from rubin_scheduler.scheduler.example import example_scheduler
from rubin_scheduler.scheduler import sim_runner
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.scheduler.utils import SchemaConverter
from rubin_scheduler.sim_archive import sim_archive, drive_sim

In [ ]:
import schedview
import schedview.app.prenight
import schedview.compute.scheduler
import schedview.collect.opsim
from schedview.plot.visitmap import BAND_COLORS

### Further preparation of the notebook

In [ ]:
pn.extension("terminal")

### Filter warnings

Several dependencies throw prodigious instances of (benign) warnings.
Suppress them to avoid poluting the executed notebook.

In [ ]:
warnings.filterwarnings(
    "ignore",
    module="astropy.time",
    message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
)
warnings.filterwarnings(
    "ignore",
    module="pandas",
    message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="divide by zero encountered in divide",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="invalid value encountered in multiply",
)
warnings.filterwarnings(
    "ignore",
    module="holoviews",
    message="Discarding nonzero nanoseconds in conversion.",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="invalid value encountered in arcsin",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="All-NaN slice encountered",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim.scheduler.utils",
    message="invalid value encountered in cast",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.scheduler.utils",
    message="invalid value encountered in cast",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.scheduler.surveys",
    message="All-NaN slice encountered",
)

## Configuration

Setting `keep_rewards` to `True` stores rewards that can be plotted in the schedview dashboards, but makes the simulation slower.

In [ ]:
keep_rewards = True

Set the date we are simulating:

In [ ]:
evening_iso8601 = "2025-05-05"

If we just use this day as the start and make the simulation duration 1 day, the begin and end of the simulation will probably begin in the middle on one night and end in the middle of the next.
Instead, find the sunset and sunrise of the night we want using the almanac, and use these to determine our start time and duration.

In [ ]:
observatory = ModelObservatory()
night_sunset_info = observatory.almanac.get_sunset_info(
    evening_date=evening_iso8601, longitude=observatory.location.lon
)

mjd_start = night_sunset_info["sun_n12_setting"]
night_duration = night_sunset_info["sunrise"] - mjd_start

Now instantiate the `ModelObservatory` and scheduler we will actually use for the simulation.

In [ ]:
observatory = ModelObservatory(mjd_start=mjd_start)

In [ ]:
scheduler = example_scheduler(mjd_start=mjd_start)
scheduler.keep_rewards = True

## Get a URI for the archive into which to save the simulation

For this sample, use a temporary directory:

In [ ]:
if True:
    archive_dir = TemporaryDirectory()
    archive_uri = lsst.resources.ResourcePath(
        archive_dir.name, forceDirectory=True
    ).geturl()

If you have a local non-temporary you want to use instead, do something like this:

In [ ]:
if False:
    archive_uri = "file:///my/directory/"

To save into the S3 bucket used by `schedview`, set both the uri and the environment variables necessary for access to it:

In [ ]:
if False:
    os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu/"
    os.environ["AWS_PROFILE"] = "prenight"
    archive_uri = "s3://rubin-scheduler-prenight/opsim/"

In both of the above cases, be sure to include the trailing `/`.

## Save the notebook in a temporary directory

In [ ]:
scratch_dir = TemporaryDirectory()
scratch_path = Path(scratch_dir.name)
notebook_fname = scratch_path.joinpath("notebook.ipynb").as_posix()

In [ ]:
%notebook $notebook_fname

## Run the simulation

In [ ]:
exec_start_time = Time.now()
results = drive_sim(
    observatory=observatory,
    scheduler=scheduler,
    archive_uri=archive_uri,
    label=f"Notebook test on {exec_start_time.iso}",
    notebook=notebook_fname,
    tags=["notebook", "devel"],
    mjd_start=mjd_start,
    survey_length=night_duration,
)

In [ ]:
sim_uri